In [1]:
import numpy as np
import pandas as pd
import time
import os
import sys
import matplotlib.pyplot as plt

## Utils

In [2]:
def read_file(filename: str):
    with open(filename) as file:
        books_in_library_with_id = []
        nb_books_in_library_with_id = []
        signup_days_for_library_with_id = []
        shipping_speed_for_library_with_id = []
        
        for idx, line in enumerate(file):
            if(idx == 0):
                # Start of the file
                line_1 = line.replace('\n', '').split(' ')
#                 print(line_1)
                nb_books = int(line_1[0])
                nb_libraries = int(line_1[1])
                nb_days = int(line_1[2])

#                 print(nb_books)
#                 print(nb_libraries)
#                 print(nb_days)
            elif(idx == 1):
                book_scores_with_id = line.replace('\n', '').split(' ')
                book_scores_with_id = list(map(int,book_scores_with_id))

            else:

                if(idx % 2 == 0):
                    
                    #### Per library section
                    line_1 = line.replace('\n', '').split(' ')
                    if(line_1[0] == ''):
                        break

                    nb_books_in_library_with_id.append(int(line_1[0]))
                    signup_days_for_library_with_id.append(int(line_1[1]))
                    shipping_speed_for_library_with_id.append(int(line_1[2]))
                    
#                     print(nb_books_in_library_with_id)
#                     print(signup_days_for_library_with_id)
#                     print(shipping_speed_for_library_with_id)
                if(idx % 2 == 1):

                    book_ids = line.replace('\n', '').split(' ')
                    if(book_ids[0] == ''):
                        break

                    book_ids = list(map(int,book_ids))
                    books_in_library_with_id.append(book_ids)
#                     print("book ids : {}".format(books_in_library_with_id))



    return nb_books, \
            nb_libraries, \
            nb_days, \
            nb_books_in_library_with_id, \
            signup_days_for_library_with_id, \
            shipping_speed_for_library_with_id, \
            book_scores_with_id ,\
            books_in_library_with_id

In [3]:
def output(lib_ids, book_ids, filename):
    with open(filename, "w") as file:
        line = str(len(lib_ids)) + "\n"
        file.write(line)
        for lib_id, ids in zip(lib_ids, book_ids):
            line1 = str(lib_id) + " " + str(len(ids)) + "\n"
            file.write(line1)
            line2 = ""
            for id in ids:
                line2 += str(id) + " "
            line2 = line2[:-1] + "\n"
            file.write(line2)

## Naive algorithm

Sort libraries by signup_days_for_library_with_id and return them with a book ordering maximizing the score

In [4]:
file_names = ["data/a_example.txt", "data/b_read_on.txt", 
              "data/c_incunabula.txt", "data/d_tough_choices.txt", 
              "data/e_so_many_books.txt", "data/f_libraries_of_the_world.txt"]

In [5]:
file_name = file_names[2]

In [65]:
def sort_books_per_library(library_list_in_order, nb_days):
    
    
    already_booked = set()
    score = 0
    
    book_shipping_sorted_for_library_with_id = []
    for library_id in library_list_in_order.copy():
        nb_days_to_ship = nb_days - signup_days_for_library_with_id[library_id]

        shipping_order = []
        current_library_shipping_speed = shipping_speed_for_library_with_id[library_id]
        index = 0
        for book_id in books_in_library_with_id[library_id]:
            
            if(nb_days_to_ship<=0):
                break
            
            if(book_id in already_booked):
                continue
                
            shipping_order.append(book_id)
            score += book_scores_with_id[book_id]
            already_booked.add(book_id)
            index += 1
            
            if(index>=current_library_shipping_speed):
                index = 0
                nb_days_to_ship -= 1
                
            
        
        if not (shipping_order == []):
            nb_days -= signup_days_for_library_with_id[library_id]
            book_shipping_sorted_for_library_with_id.append(shipping_order)
        else:
            library_list_in_order.remove(library_id)
        '''print(nb_days)'''
    return book_shipping_sorted_for_library_with_id, library_list_in_order, score

In [66]:
file_names = ["data/a_example.txt", "data/b_read_on.txt", 
              "data/c_incunabula.txt", "data/d_tough_choices.txt", 
              "data/e_so_many_books.txt", "data/f_libraries_of_the_world.txt"]

In [67]:
def sort_books(books_in_library_with_id):
    
    
    '''    book_scores_index_sorted = list(reversed(sorted(range(len(book_scores_with_id)), key=book_scores_with_id.__getitem__)))
    '''    
    
    for library_id in range(nb_libraries):
        key = []
        for book_id in books_in_library_with_id[library_id]:
            key.append(book_scores_with_id[book_id])
        
        import numpy
        sorted_books_in_library_with_id = numpy.array(books_in_library_with_id[library_id])
        key = numpy.array(key)
        inds = key.argsort()
        books_in_library_with_id[library_id] = list(reversed(sorted_books_in_library_with_id[inds].tolist()))
        
        
       
    return books_in_library_with_id

In [68]:
sub_names = ['a.txt', 'b.txt', 'c.txt', 'd.txt', 'e.txt', 'f.txt']

final_score = 0
_id_ = 0
for file_name in file_names:
    print(file_name)
    start_time = time.time()
    nb_books , \
	nb_libraries, \
	nb_days, \
	nb_books_in_library_with_id, \
	signup_days_for_library_with_id, \
	shipping_speed_for_library_with_id, \
	book_scores_with_id ,\
	books_in_library_with_id  = read_file(file_name)
    
    books_in_library_with_id = sort_books(books_in_library_with_id)


    
    tmp = np.sort(signup_days_for_library_with_id)
    indices = np.argsort(signup_days_for_library_with_id)
    
    cumsum = np.cumsum(tmp).tolist()
    
    lib_to_use = []
    count = 0

    while True:

        if(len(cumsum) <= count):
            break
        if(cumsum[count]>nb_days):
            break

        lib_to_use.append(indices[count])
        count +=1
        
    out, lib_to_use, score = sort_books_per_library(lib_to_use, nb_days)
    print(score)
    final_score += score
    output(lib_to_use, out, sub_names[_id_])
        
    print(time.time() - start_time)
    '''out, lib_to_use = sort_books_per_library(lib_to_use)
    output(lib_to_use, out, sub_names[_id_])'''
    _id_+=1
    
print(final_score)
    

data/a_example.txt
21
0.010924339294433594
data/b_read_on.txt
5822900
0.11234211921691895
data/c_incunabula.txt
5467966
0.22726678848266602
data/d_tough_choices.txt
4103905
0.5865111351013184
data/e_so_many_books.txt
4253309
0.39175844192504883
data/f_libraries_of_the_world.txt
2690353
0.3590424060821533
22338454


In [63]:
nb_days

700

In [64]:
from sets import Set

ImportError: cannot import name 'Set'

In [48]:
22338454 - 22255241

83213

In [70]:
score = 0
for a in out:
    for x in a:
        score += book_scores_with_id[x]
print(score)

2690353
